In [6]:
from mpl_toolkits.mplot3d import axes3d
import numpy as np
import math
import sys
import copy
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')


def unitVector(vector):
    """
        Returns the unit vector of a given input vector.
        Params:
            vector -> input vector.
        Returns:
            numpy.array().
    """

    
    return vector / np.linalg.norm(vector)


class Segment2D:

    """
        A part of the FabrikSolver2D to store a part of an inverse kinematics chain.
    """

    def __init__(self, referenceX, referenceY, length, angle):
        """
            Params:
                referenceX -> x component of the reference point.
                referenceY -> y component of the reference point.
                length -> length of the segemnt.
                angle -> initial angle of the segment.
        """

        self.angle = angle

        # Store the length of the segment.
        self.length = length

        
        deltaX = math.cos(math.radians(angle)) * length
        deltaY = math.sin(math.radians(angle)) * length

        newX = referenceX + deltaX
        newY = referenceY + deltaY

        self.point = np.array([newX, newY])

    def setPoint(self, a, b, reference):
        # TODO: add high level function for updating point.
        pass


class FabrikSolver2D:
    """
        An inverse kinematics solver in 2D. Uses the Fabrik Inverse Kinematics Algorithm.
    """

    def __init__(self, baseX=0, baseY=0, marginOfError=0.01):
        """
            Params:
                baseX -> x component of the base.
                baseY -> y coördinate of the base.
                marginOfError -> the margin of error for the algorithm.
        """

        self.basePoint = np.array([baseX, baseY])

        self.segments = []

        self.history = []

        self.armLength = 0

        #
        self.marginOfError = marginOfError

    def addSegment(self, length, angle):
        """
            Add new segment to chain with respect to the last segment.
            Params:
                length -> length of the segment.
                angle -> initial angle of the segment.
        """

        if len(self.segments) > 0:

            segment = Segment2D(
                self.segments[-1].point[0], self.segments[-1].point[1], length, angle + self.segments[-1].angle)
        else:
            segment = Segment2D(
                self.basePoint[0], self.basePoint[1], length, angle)

        self.armLength += segment.length

        self.segments.append(segment)

    def isReachable(self, targetX, targetY):
        """  
            Check if a point in space is reachable by the end-effector.
            Params:
                targetX -> the target x coördinate to check.
                targetY -> the target y coördinate to check.
            Returns:
                Boolean.  
        """

        if np.linalg.norm(self.basePoint - np.array([targetX, targetY])) < self.armLength:
            return True
        return False

    def inMarginOfError(self, targetX, targetY):
        """  
            Check if the distance of a point in space and the end-effector is smaller than the margin of error.
            Params:
                targetX -> the target x coördinate to check.
                targetY -> the target y coördinate to check.
                targetZ -> the target z coördinate to check. 
            Returns:
                Boolean.  
        """

        if np.linalg.norm(self.segments[-1].point - np.array([targetX, targetY])) < self.marginOfError:
            return True
        return False

    def iterate(self, targetX, targetY):
        """ 
            Do one iteration of the fabrik algorithm. Used in the compute function. 
            Use in simulations or other systems who require motion that converges over time.  
            Params:
                targetX -> the target x coördinate to move to.
                targetY -> the target y coördinate to move to.
        """

        target = np.array([targetX, targetY])

        # Forward.
        for i in range(len(self.segments) - 1, 0, -1):

            if i == len(self.segments) - 1:
                self.segments[i-1].point = (unitVector(
                    self.segments[i-1].point - target) * self.segments[i].length) + target
            else:
                self.segments[i-1].point = (unitVector(self.segments[i-1].point -
                                                       self.segments[i].point) * self.segments[i].length) + self.segments[i].point

         # Backward.
        for i in range(len(self.segments)):
            if i == 0:
                self.segments[i].point = (unitVector(
                    self.segments[i].point - self.basePoint) * self.segments[i].length) + self.basePoint

            elif i == len(self.segments) - 1:
                self.segments[i].point = (unitVector(
                    self.segments[i-1].point - target) * self.segments[i].length * -1) + self.segments[i-1].point

            else:
                self.segments[i].point = (unitVector(
                    self.segments[i].point - self.segments[i-1].point) * self.segments[i].length) + self.segments[i-1].point

    def compute(self, targetX, targetY):
        """  
            Iterate the fabrik algoritm until the distance from the end-effector to the target is within the margin of error.
            Params:
                targetX -> the target x coördinate to move to.
                targetY -> the target x coördinate to move to.
        """

        self.history.append(copy.deepcopy(self.segments))
        if self.isReachable(targetX, targetY):
            while not self.inMarginOfError(targetX, targetY):
                self.iterate(targetX, targetY)
                self.history.append(copy.deepcopy(self.segments))
        else:
            print('Target not reachable.')

    def plot(self, segments=None, save=False, name="graph", xMin=-300, xMax=300, yMin=-300, yMax=300):
        """  
            Plot the chain.
            Params:
                save -> choose to save the plot to a file.
                name -> give the plot a name.
                xMin -> the left bound of the plot.
                xMax -> the right bound of the plot.
                yMin -> the low bouwnd of the plot.
                yMax -> the hight bound of the plot.
        """

        if segments == None:
            segments = self.segments

        for i in range(len(segments)):

            plt.plot([segments[i].point[0]], [
                     segments[i].point[1]], 'ro')

            plt.text(segments[i].point[0], segments[i].point[1] + 1, '(x:{}, y:{})'.format(
                int(segments[i].point[0]), int(segments[i].point[1])))

        plt.plot([self.basePoint[0]], [self.basePoint[1]], 'bo')
        plt.text(self.basePoint[0], self.basePoint[1], 'Base')

        plt.axis([xMin, xMax, yMin, yMax])
        plt.grid(True)

        if save == True:
            plt.savefig('{}.png'.format(name))

        plt.show(block=True)

In [16]:
# from fabric import FabrikSolver2D
import matplotlib.pyplot as plt
import math


def Angle(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    dX = x2 - x1
    dY = y2 - y1
    rads = math.atan2(-dY, dX) 
    return math.degrees(rads)


def Len(line):
    return ((line[0][0] - line[1][0])**2 + (line[0][1] - line[1][1])**2)**(0.5)


def AngleFromLines(lines):

    for line1 in lines:
        for line2 in lines:
            if line1 == line2:
                continue
            line1StPnt, line1EndPnt = line1
            line2StPnt, line2EndPnt = line2
            angle1 = Angle(line1StPnt, line1EndPnt)
            angle2 = Angle(line2StPnt, line2EndPnt)
            angle = abs(angle1 - angle2)
            return angle

def plotIterations(history):
    for i in range(len(history)):
        x = [0]
        y = [0]
        for segment in history[i]:
            x.append(segment.point[0])
            y.append(segment.point[1])

        plt.plot(x, y, label=("line " + str(i + 1)))
    plt.xlabel('x - axis')
    plt.ylabel('y - axis') 
    plt.title('Iterations history')
    plt.legend()
    plt.show()
lines = []
totalLinks = int(input("Total links: "))
for i in range(totalLinks):
    text = input("Enter coord[" + str(i) + "]: ")
    coord = tuple(float(x) for x in text.split())
    lastCoord = (0, 0)
    if (len(lines) > 0):
        lastCoord = lines[-1][1]
    lines.append(((lastCoord, coord)))
arm = FabrikSolver2D()
for i in range(len(lines)):
    linkLen = Len(lines[i])
    lastLine = ((-1, 0), (0, 0))
    if i != 0:
        lastLine = lines[i - 1]
    angle = AngleFromLines([lastLine, lines[i]])
    arm.addSegment(linkLen, angle)

final_coord = tuple(float(x) for x in input(
    "Enter end effector final position: ").split())

arm.compute(final_coord[0], final_coord[1])

print("total iterations: ", len(arm.history) - 1)

for segments in arm.history:
    text = ''
    for segment in segments:
        text += "[" + str(segment.point[0]) + ", " + \
            str(segment.point[1]) + "], "
    print(text[0:-2])
plotIterations(arm.history)

Total links: 2
Enter coord[0]: 0.1
Enter coord[1]: 0.2


IndexError: tuple index out of range

In [13]:
# A graphical user interface to get data of a Denavit-Hartenberg parameters using python

# tkinter module is the standard Python interface to the GUI toolkit
import tkinter as tkin

# Declaring global variables
get_data = []
no_of_rows = 0


# Defining a method for invalid inputs from user
def error_handel(window):

    # once ok button is pressed in error window it is destroyed
    window.destroy()

    # the main window is brought back
    screen.deiconify()


# Defing 'save_data' a method for saving inserted data from table
def save_data():

    # variables 'DH_Parameters' will contain the values of angel about z, displacement about z, angel about x and displacement along x.
    # in four lists.
    DH_Parameters = []

    # 'get_data' is mapped to every rows and columns of the table
    # 'values' represent each columns of the table
    for values in get_data:

        # 'parameter' will be storing all values of a particular parameter at each iteration
        parameter = []

        # 'individual_slot' points to the individual slots of the table
        for individual_slot in values:
            try:
                # 'individual_slot.get()' method returns the value entered in the slot
                parameter.append(float(individual_slot.get()))

            # exception for invalid values entered
            except ValueError:

                # another GUI window is defined 'error'
                error = tkin.Tk()
                error.title("ERROR")
                error.geometry("+100+100")

                # hiding the main window until the ok Button is pressed in exception window
                screen.withdraw()

                # displaying the error message
                tkin.Label(error, text = "Enter a valid parameter value", font = font_data).grid(row = 0, column =0)
                ok_button = tkin.Button(error, text = "ok",font = font_data,bg="red", width = 3,command = error_handel)

                # passing the exception window variable as val
                ok_button ['command'] = lambda val = error:error_handel(val)
                ok_button.grid(row = 1, column = 0)
                error.mainloop()

        # now 'parameter' contains all values of a particular parameter entered in the table form GUI
        DH_Parameters.append(parameter)
        # screen.destroy()

    # 'plot_dh' is method which takes DH Parameters as input and convert it to 3D coordinates, that is x, y, and z points and plot it in a 3D graph
    print(DH_Parameters)
    # plot_dh(DH_Parameters)


# Defining 'add_row' to add a new row for DH Parameters everytime user presses '+' button
def add_row():
    global no_of_rows

    # Delete the button already present beside every row, that is '+' and 'save' buttons
    add.destroy()
    save.destroy()

    # keep count number of rows
    no_of_rows += 1

    # serail number is auto generated using the following method
    tkin.Label(screen, text = str(no_of_rows)+".", width = width_of_slot, font = font_data).grid(row = no_of_rows, column = 0)

    # 'mapping' is a variable used to map the individual slot and then it is appended to 'get_data' list
    mapping = []

    # for loop for four columns of every row
    for individual_slot in range(0,4):

        # method to create slots to enter variables
        mapping.append(tkin.Entry(screen, width = width_of_slot, font = font_data))

        # setting default values as '0'
        mapping[individual_slot].insert(0,"0")

        # positoning of the slot
        mapping[individual_slot].grid(row = no_of_rows, column = 1+individual_slot)
    get_data.append(mapping)

    # defining the deleted buttons in a new positon
    global add
    add = tkin.Button(screen, text = "+",font = font_data,bg="lightblue", width = 3,command = add_row)
    add.grid(row = no_of_rows, column = 5)
    global save
    save = tkin.Button(screen, text = "Plot",font = font_data,bg="lightgreen", width = 3,command = save_data)
    save.grid(row = no_of_rows, column = 6)


# main function
def DH_Table_Plotter_main():
    width_of_slot = 8
    font_data = 12
    # defining a master variable for tkinter gui methode
    screen = tkin.Tk()

    # positon of the window
    screen.geometry("+0+10")

    # title of the window
    screen.title("DH Table Convertor")

    # initial lables of the GUI
    tkin.Label(screen,text = "Enter The values ")
    tkin.Label(screen,text = "Link No.",width=width_of_slot,font = font_data).grid(row = 0, column = 0)
    tkin.Label(screen,text = "Z_dis",width=width_of_slot,font = font_data).grid(row = 0, column = 1)
    tkin.Label(screen,text = "Theta",width=width_of_slot,font = font_data).grid(row = 0, column = 2)
    tkin.Label(screen,text = "X_dis",width=width_of_slot,font = font_data).grid(row = 0, column = 3)
    tkin.Label(screen,text = "Alpha",width=width_of_slot,font = font_data).grid(row = 0, column = 4)

    # defining both buttons for the first time
    add = tkin.Button(screen, text = "+", font = font_data, width = 3, bg="lightblue", command = add_row)
    add.grid(row = no_of_rows, column = 5)
    save = tkin.Button(screen, text = "Plot",font = font_data, bg="lightgreen", width = 3, command = save_data)
    save.grid(row = no_of_rows, column = 6)

    # calling mainloop
    screen.mainloop()

SyntaxError: name 'add' is used prior to global declaration (<ipython-input-13-1f84bc3f2c6a>, line 101)